In [1]:
# Imports
from typing import Union
import pandas as pd
import requests
from datetime import datetime
import pytse_client as tse
import fetch
import config
from utils import debug_log, data_log, ar_to_fa_series, ar_to_fa, fa_to_ar, fa_to_ar_series, flatten_json

pd.set_option('display.max_columns', 30) # pd.options.display.max_columns = 3
pd.set_option('display.max_rows', 7) # pd.options.display.max_rows = 3
pd.set_option('display.float_format', '{:.2f}'.format) # = pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_colwidth = 10

#### Common Parameters

In [2]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Referer": "http://main.tsetmc.com/StaticContent/WebServiceHelp",
}
cookie_jar = {"ASP.NET_SessionId": "wa40en1alwxzjnqehjntrv5j"}
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']
# currentDateTime = datetime.now().strftime("%Y-%m-%d %H-%M-%S %p")

last_date = fetch.api_last_possible_deven()
instruments_df = fetch.get_instruments()
identities_df = fetch.get_identities()

insCodes = instruments_df.index
insCodes_indices_excluded = list(instruments_df[instruments_df['cComVal'] != '6'].index)

print(f'Last Date: {last_date}')
print('Instruments: ', len(insCodes))
print('Instruments (Excluding Indices): ', len(insCodes_indices_excluded))
print('Indices: ', len(insCodes) - len(insCodes_indices_excluded))

Last Date: 20230913
Instruments:  2304
Instruments (Excluding Indices):  2197
Indices:  107


In [4]:
## Getting Prices
daily_prices = fetch.get_quotes('all')
print(f'{len(daily_prices.keys())} Tickers have history.')

1935 Tickers have history.


## Categorization

In [36]:
def filter_tickers_by_industry(cSecVal: int) -> list:
    return list(identities_df[identities_df['sector_cSecVal'] == cSecVal].index)

In [37]:
khodroi = filter_tickers_by_industry(34)

## Volume Filter

In [39]:
def volume_filter(insCodes: list) -> list:
    
    daily_prices = fetch.get_quotes(insCodes)
    
    list_of_dfs = []
    ticker_names= []
    # TODO: درست کردن فیلتر «نمادهای معامله شده امروز/روز آخر کاری»ا
    for insCode in insCodes:
        last_row = daily_prices[insCode].iloc[-1]
        if (
            daily_prices[insCode].index[-1] == int(last_date) and
            last_row['<VOL>'] > last_row['<VOL> MA 3'] and 
            last_row['<VOL>'] > last_row['<VOL> MA 5'] and
            last_row['<VOL>'] > last_row['<VOL> MA 9'] and
            last_row['<VOL>'] > last_row['<VOL> MA 26'] and
            last_row['<VOL>'] > last_row['<VOL> MA 52'] and
            last_row['<VOL>'] > last_row['<VOL> MA 90']
        ):
            list_of_dfs.append(daily_prices[insCode])
            ticker_names.append(insCode)
            debug_log.info(f'{insCode} Filtered For Volume.')
    print(len(ticker_names),'\n')
    return ticker_names


In [41]:
volume_filter(list(daily_prices.keys()))

77 



['1358190916156744',
 '1931037287459558',
 '2161110547458064',
 '3493306453706327',
 '3654864906585643',
 '3846143218462419',
 '4151098369137517',
 '4247709727327181',
 '5054819322815158',
 '5599691633622269',
 '8646067353086740',
 '9536587154100457',
 '9546211590069761',
 '9698674686691945',
 '11964419322927535',
 '13281937213456378',
 '16040900750729921',
 '16959429956899455',
 '17226661368470120',
 '17617474823279712',
 '19367527798307032',
 '19828734979381742',
 '22382156782768756',
 '22667016906590506',
 '23175320865252772',
 '24079409192818584',
 '24085906177899789',
 '26543014712914772',
 '27797446447955609',
 '28328083629154916',
 '30282299500988269',
 '30974710508383145',
 '31569200988534548',
 '35424116338766901',
 '39116664428676213',
 '42696242981550091',
 '43267179898797137',
 '45205530868811305',
 '47563321799863211',
 '48241092863917835',
 '48557136219738552',
 '48619517949257749',
 '49054891736433700',
 '52846735736632974',
 '53251602435454519',
 '53686258677793038',
 '